### Langchain: Q & A Over Documents 

Example tool to query product catalog.



In [22]:
# !pip install --upgrade langchain
# !pip install --upgrade bs4

In [23]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [24]:

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
import langchain
langchain.verbose = False


In [25]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file, encoding='utf-8')

In [26]:
from langchain.indexes import VectorstoreIndexCreator

In [27]:
# !pip install docarray

In [28]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [29]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [30]:
response = index.query(query)

In [31]:
display(Markdown(response))



| Name | Description |
| --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated sun protection, 100% polyester fabric, wrinkle-resistant, front and back cape venting, two front bellows pockets |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ rated sun protection, 52% polyester and 48% nylon fabric, machine washable and dryable, front and back cape venting, two front bellows pockets |
| Sun Shield Shirt by | UPF 50+ rated sun protection, 78% nylon and 22% Lycra Xtra Life fiber fabric, wicks moisture for quick-drying comfort, abrasion resistant, imported |
| Men's TropicVibe Shirt, Short-Sleeve | UPF 50+ rated sun protection, 71% Nylon and 29% Polyester fabric, machine wash and dry, wrinkle resistant, front and back cape venting, two front bellows pockets |

All of the shirts listed provide UPF 50+ rated sun protection, blocking 98% of the sun's harmful rays. The fabrics used vary, but all are machine washable and dryable. Additional

In [32]:
loader = CSVLoader(file_path=file, encoding='utf-8')

In [33]:
docs = loader.load()

In [34]:
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \r\n\r\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \r\n\r\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \r\n\r\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \r\n\r\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [35]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [36]:
embed = embeddings.embed_query("Hi my name is Chris")

In [37]:
print(len(embed))

1536


In [38]:
print(embed[:5])

[-0.017098527401685715, 0.007435472682118416, -0.015752186998724937, -0.037868890911340714, -0.019558658823370934]


In [39]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [40]:
query = "Please suggest a shirt with sunblocking"

In [41]:
docs = db.similarity_search(query)

In [42]:
len(docs)

4

In [43]:
docs[0]

Document(page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \r\n\r\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\r\n\r\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\r\n\r\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\r\n\r\nSun Protection That Won\'t Wear Off\r\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255})

In [44]:
retriever = db.as_retriever()

In [45]:
llm = ChatOpenAI(temperature = 0.0)

In [46]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [47]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.")

In [48]:
display(Markdown(response))

| Shirt Name | Description | 
| --- | --- | 
| Sun Shield Shirt | High-performance sun shirt with UPF 50+ sun protection, moisture-wicking, and abrasion-resistant fabric. Recommended by The Skin Cancer Foundation. | 
| Men's Plaid Tropic Shirt | Ultracomfortable shirt with UPF 50+ sun protection, wrinkle-free fabric, and front and back cape venting. Great for extended travel. | 
| Men's TropicVibe Shirt | Men's sun-protection shirt with built-in UPF 50+ and a lightweight feel. Features wrinkle-resistant fabric, front and back cape venting, and two front bellows pockets. | 
| Men's Tropical Plaid Short-Sleeve Shirt | Lightest hot-weather shirt with UPF 50+ sun protection, traditional fit, and front and back cape venting. Made of wrinkle-resistant 100% polyester. | 

Each shirt provides UPF 50+ sun protection, blocking 98% of the sun's harmful rays. The Sun Shield Shirt is recommended by The Skin Cancer Foundation and has moisture-wicking and abrasion-resistant fabric. The Men's Plaid Tropic Shirt is great for extended travel and has wrinkle-free fabric and front and back cape venting. The Men's TropicVibe Shirt has a lightweight feel and features wrinkle-resistant fabric and front and back cape venting. The Men's Tropical Plaid Short-Sleeve Shirt has a traditional fit and is made of wrinkle-resistant 100% polyester.

In [49]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [50]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [51]:
response = qa_stuff.run(query)



> Entering new  chain...

> Finished chain.


In [52]:
display(Markdown(response))

| Shirt Name | Description |
| --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | Rated UPF 50+ for superior protection from the sun's UV rays. Made of 100% polyester and is wrinkle-resistant. With front and back cape venting that lets in cool breezes and two front bellows pockets. Provides the highest rated sun protection possible. |
| Men's Plaid Tropic Shirt, Short-Sleeve | Rated to UPF 50+, helping you stay cool and dry. Made with 52% polyester and 48% nylon, this shirt is machine washable and dryable. Additional features include front and back cape venting, two front bellows pockets and an imported design. With UPF 50+ coverage, you can limit sun exposure and feel secure with the highest rated sun protection available. |
| Sun Shield Shirt by | High-performance sun shirt is guaranteed to protect from harmful UV rays. Made of 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. |
| Men's TropicVibe Shirt, Short-Sleeve | Sun-protection shirt with built-in UPF 50+. Made of Shell: 71% Nylon, 29% Polyester. Lining: 100% Polyester knit mesh. UPF 50+ rated – the highest rated sun protection possible. Wrinkle resistant. Front and back cape venting lets in cool breezes. Two front bellows pockets. |

The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant. It is rated UPF 50+ for superior protection from the sun's UV rays. The Men's Plaid Tropic Shirt, Short-Sleeve is made with 52% polyester and 48% nylon, and is machine washable and dryable. It is rated to UPF 50+ and has front and back cape venting, two front bellows pockets, and an imported design. The Sun Shield Shirt by is made of 78% nylon, 22% Lycra Xtra Life fiber, and is UPF 50+ rated. It wicks moisture for quick-drying comfort, fits comfortably over your favorite swimsuit, and is abrasion-resistant. The Men's TropicVibe Shirt, Short-Sleeve is a sun-protection shirt with built-in UPF 50+. It is made of Shell: 71% Nylon, 29% Polyester. Lining: 100% Polyester knit mesh, and is UPF 50+ rated. It is wrinkle-resistant, has front and back cape venting that lets in cool breezes, and two front bellows pockets.

In [53]:
response = index.query(query, llm=llm)

In [54]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])